In [27]:
import pandas as pd 
import numpy as np

from re import sub
from decimal import Decimal
from datetime import datetime

import json 
import csv 

# Opening JSON file and loading the data 
# into the variable data 
with open('kijiji-output.json') as json_file: 
    data = json.load(json_file) 

In [28]:
# Keywords
bedId = ["bed", "bedroom", "bdrm"]
bathId = ["bath"]
minuteId = ["min", "mins", "minutes", "minute", "minutes"]

bad = ["food", "promo", "commercial"]

textDic = {"one": 1, "two": 2, "three": 3, "four": 4, "five":5, 
           "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10}

In [25]:
# Build the dataframe
clean_df = pd.DataFrame(columns=list(data[0].keys()))
bad_words = ["food"]

i = 0

for line in data:
    # Checks if it's a non rental posting
    if any(sub in data[i]['title'].lower() for sub in bad):
        print("WATCH OUT")
        i += 1
        continue
    
    # The dirty string
    dirty = list(line.values())
    
    # For money
    money = 0
    try:
        money = round(Decimal(sub(r'[^\d.]', '', dirty[0])))
    except:
        money = np.nan
        
    # For title
    title = dirty[1]
        
    # For distance
    distance = np.nan
    lTitle = title.lower().replace("-", " ")
    ltSplit = lTitle.split()
    
    for j in range(len(ltSplit)):

        if ltSplit[j] in minuteId:
            if ltSplit[j] == "minutes":
                distance = 3
                break
            
            prev = ltSplit[j-1]

            try:
                distance = int(prev)
            except:    
                if prev in textDic:
                    distance = textDic[prev]
                else:
                    print("FAILED Distance calc")
                    distance = np.nan
                    
            break
        
    # For date
    date = dirty[3].replace("/", "")
    print(data[i])
    # Makes it today's date if it got posted recently
    if len(date) == 0 or date[0] == "<":
        date = datetime.today().strftime('%d%m%Y')

    
    clean_df.loc[i] = [money, data[i]['title'], distance,
                       date, data[i]['location'], data[i]['description']]
    
    i += 1

{'price': '$2,775.00', 'title': '3 BED 3 BATH - PRIVATE ENSUITE BATHROOM FOR EVERY RESIDENT!', 'distance': '', 'date-posted': '07/01/2021', 'location': 'Kingston', 'description': 'Starting at $925/Bedroom per Month Private ensuite bathroom for every resident! Designed for safety and social distance âœ“ Private entrances âœ“ Private bathrooms CURRENTLY UNDER CONSTRUCTION - LEASING ...'}
{'price': '$2,455.00', 'title': "MODERN 3-Bedroom Apartment Downtown - 5 Mins to Queen's May 2021", 'distance': '', 'date-posted': '17/01/2021', 'location': 'Kingston', 'description': "3D virtual tour available â€“ contact for details! http://mackinnondev.ca/rental/122-sydenham/ 3-bedroom apartment on the third floor of a professionally-maintained brick complex in heart of downtown ...                                                                                                                                                                                                                               

{'price': '$2,775.00', 'title': '3 BED 3 BATH - PRIVATE ENSUITE BATHROOM FOR EVERY RESIDENT!', 'distance': '', 'date-posted': '07/01/2021', 'location': 'Kingston', 'description': 'Starting at $925/Bedroom per Month Private ensuite bathroom for every resident! Designed for safety and social distance âœ“ Private entrances âœ“ Private bathrooms CURRENTLY UNDER CONSTRUCTION - LEASING ...'}
{'price': '$2,625.00', 'title': 'Renovated from top to bottom! 3 Bedroom - Available May 1st', 'distance': '', 'date-posted': '07/01/2021', 'location': 'Kingston', 'description': 'It is a beautiful spacious unit in a great location. This property has all stainless steel appliances and an in-suite washer/dryer. All our properties have keyless entry, a full-time maintenance ...'}
{'price': '$650.00', 'title': "Wanted:                            WANTED: 4 Bedroom House Close to Queen's Main Campus", 'distance': '', 'date-posted': '07/01/2021', 'location': 'Kingston', 'description': 'We are a group of 4 girl

In [26]:
clean_df

,price,title,distance,date-posted,location,description
0,2775,3 BED 3 BATH - PRIVATE ENSUITE BATHROOM FOR EV...,NaN,07012021,Kingston,Starting at $925/Bedroom per Month Private ens...
1,2455,MODERN 3-Bedroom Apartment Downtown - 5 Mins t...,5,17012021,Kingston,3D virtual tour available â€“ contact for deta...
2,2625,Renovated from top to bottom! 3 Bedroom - Avai...,NaN,07012021,Kingston,It is a beautiful spacious unit in a great loc...
3,1950,"2 Bedroom, Available May 1st, 2021 - 10 MINS F...",10,14012021,Kingston,"Just 10 minutes to Queen's University, and ste..."
4,2673,RENOVATED 3-Bedroom Apartment - 5 Mins to Quee...,5,17012021,Kingston,"Completely renovated, 3-bedroom apartment is o..."
...,...,...,...,...,...,...
66,590,ASAP - Looking for Sublet for Jan 1st utilitie...,NaN,29112020,Kingston,I'm looking for someone to sublet my room or t...
67,1900,WANTED: Apartment/house near Queen's University,NaN,29112020,Kingston,"Hello, we are two female students from the Fac..."
68,600,Single Room for rent all inclusive 600.00,NaN,28112020,Kingston,"One second floor bedroom for Rent 600 , includ..."
69,570,Looking for 2 Student/Young Professional Room...,NaN,24112020,Kingston,Looking for two students/young professionals r...


In [30]:
## now we will open a file for writing 
#data_file = open('data_file.csv', 'w') 
#
## create the csv writer object 
#csv_writer = csv.writer(clean_df) 
#    
#data_file.close()

In [31]:
clean_df.to_csv(r'C:\\Eduard\\Projects\\GitHub\\QHacks-2021\\ml\\housing_data.csv', index = False)